### Global temperatures
* Source 
   * 8 data sets from GHCN-Monthly Version 3 (ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/v3/)
        4 avg datasets
        2 min datasets
        2 max datasets
    
   * Selection: We only used 1 min and 1 max dataset.
        ./Resources/noaa/ghcn/csv/ghcnm.tmax.v3.3.0.20170708.qca.dat.csv
        ./Resources/noaa/ghcn/csv/ghcnm.tmin.v3.3.0.20170708.qca.dat.csv

In [1]:
import pandas as pd
from datetime import date, datetime

### DATA FORMAT

          Variable          Columns      Type
          --------          -------      ----

          ID                 1-11        Integer
          YEAR              12-15        Integer
          ELEMENT           16-19        Character
          VALUE1            20-24        Integer
          DMFLAG1           25-25        Character
          QCFLAG1           26-26        Character
          DSFLAG1           27-27        Character
            .                 .             .
            .                 .             .
            .                 .             .
          VALUE12          108-112       Integer
          DMFLAG12         113-113       Character
          QCFLAG12         114-114       Character
          DSFLAG12         115-115       Character

### Variable Definitions:

          ID: 11 digit identifier, digits 1-3=Country Code, digits 4-8 represent
              the WMO id if the station is a WMO station.  It is a WMO station if
              digits 9-11="000".

          YEAR: 4 digit year of the station record.
 
          ELEMENT: element type, monthly mean temperature="TAVG"
                                 monthly maximum temperature="TMAX"
                                 monthly minimum temperature="TMIN"

          VALUE: monthly value (MISSING=-9999).  Temperature values are in
                 hundredths of a degree Celsius, but are expressed as whole
                 integers (e.g. divide by 100.0 to get whole degrees Celsius).

          DMFLAG: data measurement flag, nine possible values:

                  blank = no measurement information applicable
                  a-i = number of days missing in calculation of monthly mean
                        temperature (currently only applies to the 1218 USHCN
                        V2 stations included within GHCNM)

          QCFLAG: quality control flag, seven possibilities within
                  quality controlled unadjusted (qcu) dataset, and 2 
                  possibilities within the quality controlled adjusted (qca) 
                  dataset.

                  Quality Controlled Unadjusted (QCU) QC Flags:
         
                  BLANK = no failure of quality control check or could not be
                          evaluated.

                  D = monthly value is part of an annual series of values that
                      are exactly the same (e.g. duplicated) within another
                      year in the station's record.

                  I = checks for internal consistency between TMAX and TMIN. 
                      Flag is set when TMIN > TMAX for a given month. 

                  L = monthly value is isolated in time within the station
                      record, and this is defined by having no immediate non-
                      missing values 18 months on either side of the value.

                  M = Manually flagged as erroneous.

                  O = monthly value that is >= 5 bi-weight standard deviations
                      from the bi-weight mean.  Bi-weight statistics are
                      calculated from a series of all non-missing values in 
                      the station's record for that particular month.

                  S = monthly value has failed spatial consistency check.
                      Any value found to be between 2.5 and 5.0 bi-weight
                      standard deviations from the bi-weight mean, is more
                      closely scrutinized by exmaining the 5 closest neighbors
                      (not to exceed 500.0 km) and determine their associated
                      distribution of respective z-scores.  At least one of 
                      the neighbor stations must have a z score with the same
                      sign as the target and its z-score must be greater than
                      or equal to the z-score listed in column B (below),
                      where column B is expressed as a function of the target
                      z-score ranges (column A). 

                                  ---------------------------- 
                                       A       |        B
                                  ----------------------------
                                    4.0 - 5.0  |       1.9
                                  ----------------------------
                                    3.0 - 4.0  |       1.8
                                  ----------------------------
                                   2.75 - 3.0  |       1.7
                                  ----------------------------
                                   2.50 - 2.75 |       1.6
                                     


                  W = monthly value is duplicated from the previous month,
                      based upon regional and spatial criteria and is only 
                      applied from the year 2000 to the present.                   

                  Quality Controlled Adjusted (QCA) QC Flags:

                  A = alternative method of adjustment used.

		  Q = value removed; original observation flagged as invalid during the automated quality control process.
 
                  M = values with a non-blank quality control flag in the "qcu"
                      dataset are set to missing the adjusted dataset and given
                      an "M" quality control flag.

                  X = pairwise algorithm removed the value because of too many
                      inhomogeneities.


          DSFLAG: data source flag for monthly value, 21 possibilities:

                  C = Monthly Climatic Data of the World (MCDW) QC completed 
                      but value is not yet published

                  D = Calculated monthly value from daily data contained within the
                      Global Historical Climatology Network Daily (GHCND) dataset.

                  G = GHCNM v2 station, that was not a v2 station that had multiple
                      time series (for the same element).

                  J = Colonial Era Archive Data
 
                  K = received by the UK Met Office

                  M = Final (Published) Monthly Climatic Data of the World 
                     (MCDW)

                  N = Netherlands, KNMI (Royal Netherlans Meteorological 
                      Institute)

                  P = CLIMAT (Data transmitted over the GTS, not yet fully 
                      processed for the MCDW)

                  U = USHCN v2

                  W = World Weather Records (WWR), 9th series 1991 through 2000 

                  Z = Datzilla (Manual/Expert Assessment)

             0 to 9 = For any station originating from GHCNM v2 that had
                      multiple time series for the same element, this flag
                      represents the 12th digit in the ID from GHCNM v2.
                      See section 2.2.2 for additional information.

                  , = CLIMAT (Data transmitted in BUFR format)                 


### Read raw csv, transform and save as csv

In [2]:
'''
function that takes in the params and transforms the data from the ghcn csv per the above format

param1: input_full_filename: the input file name
param2: element: a string with the value "TMAX" or "TMIN"
param3: header: a list of column headers
param4: output_full_filename: the output filename
'''
def read_transform_saveoutput(input_full_filename, element, header, output_full_filename):
    
    #read in the csv, adding a header row per the data spec.
    df = pd.read_csv(input_full_filename, header=None, names=header)

    #we just want the value columns, 1 for each month
    df = df[['YEAR'
        , 'VALUE1', 'VALUE2', 'VALUE3'
        , 'VALUE4', 'VALUE5', 'VALUE6'
        , 'VALUE7', 'VALUE8', 'VALUE9'
        , 'VALUE10', 'VALUE11', 'VALUE12'
       ]]

    #rename the value columns to represent the month
    df.rename(columns={
        "VALUE1": "1"
        ,"VALUE2": "2"
        ,"VALUE3": "3"
        ,"VALUE4": "4"
        ,"VALUE5": "5"
        ,"VALUE6": "6"
        ,"VALUE7": "7"
        ,"VALUE8": "8"
        ,"VALUE9": "9"
        ,"VALUE10": "10"
        ,"VALUE11": "11"
        ,"VALUE12": "12"
    }, inplace=True)

    #Set the year as the index
    df.set_index('YEAR', inplace=True)

    #unpivot the data 
    df = df.unstack().reset_index(name='value')
    df.rename(columns={"level_0": "Month", "value": element}, inplace=True)

    #convert the month to an integer data type
    df["Month"] = pd.to_numeric(df["Month"])

    #add a new date column
    df["date_time"] = df.apply(lambda row:
                              datetime(row.YEAR, row.Month, 1), 
                              axis=1)
    
    df["date_time"] = pd.to_datetime(df["date_time"])
    
    #save the transformed data to a csv for should it be required for later use.
    df.to_csv(output_full_filename, index_label='id')

In [3]:
header = ['ID', 'YEAR', 'ELEMENT', 
              'VALUE1', 'DMFLAG1', 'QCFLAG1', 'DSFLAG1',
              'VALUE2', 'DMFLAG2', 'QCFLAG2', 'DSFLAG2',
              'VALUE3', 'DMFLAG3', 'QCFLAG3', 'DSFLAG3',
              'VALUE4', 'DMFLAG4', 'QCFLAG4', 'DSFLAG4',
              'VALUE5', 'DMFLAG5', 'QCFLAG5', 'DSFLAG5',
              'VALUE6', 'DMFLAG6', 'QCFLAG6', 'DSFLAG6',
              'VALUE7', 'DMFLAG7', 'QCFLAG7', 'DSFLAG7',
              'VALUE8', 'DMFLAG8', 'QCFLAG8', 'DSFLAG8',
              'VALUE9', 'DMFLAG9', 'QCFLAG9', 'DSFLAG9',
              'VALUE10', 'DMFLAG10', 'QCFLAG10', 'DSFLAG10',
              'VALUE11', 'DMFLAG11', 'QCFLAG11', 'DSFLAG11',
              'VALUE12', 'DMFLAG12', 'QCFLAG12', 'DSFLAG12'
         ]

In [4]:
input_full_filename = './Resources/noaa/ghcn/csv/ghcnm.tmax.v3.3.0.20170708.qca.dat.csv'
element = 'TMAX'
output_full_filename = './Resources/noaa/ghcn/csv/output/tmax_tranformed_unfilterd.csv'

read_transform_saveoutput(input_full_filename, element, header, output_full_filename)

In [5]:
input_full_filename = './Resources/noaa/ghcn/csv/ghcnm.tmin.v3.3.0.20170708.qca.dat.csv'
element = 'TMIN'
output_full_filename = './Resources/noaa/ghcn/csv/output/tmin_tranformed_unfilterd.csv'

read_transform_saveoutput(input_full_filename, element, header, output_full_filename)

### Read transformed csv and group by YEAR for TMAX and TMIN

In [6]:
'''
function that takes in the tranformed csv and then groups by the TMAX or TMIN and 
returns a dataframe
param1: element: a string with the value "TMAX" or "TMIN"
'''
def read_and_groupby(element):
    filename = f'./Resources/noaa/ghcn/csv/output/{element}_tranformed_unfilterd.csv'
    df = pd.read_csv(filename)
    
    #dropping values that are not valid
    df.drop(df[df[element] == -9999].index, inplace=True)
    
    df["date_time"] = pd.to_datetime(df["date_time"])
    df = df.groupby(pd.Grouper(key='date_time', freq='Y')).agg({element: element[1:].lower()})

    df[element] = df[element]/100
    return df

In [7]:
#get TMAX dataset grouped by Year
tmax_df = read_and_groupby('TMAX')
tmax_df.head()

,TMAX
date_time,
1840-12-31,23.74
1841-12-31,25.14
1842-12-31,24.74
1843-12-31,24.84
1844-12-31,25.14


In [8]:
#get TMIN dataset grouped by Year
tmin_df = read_and_groupby('TMIN')
tmin_df.head()

,TMIN
date_time,
1840-12-31,-8.31
1841-12-31,-9.21
1842-12-31,-7.11
1843-12-31,-12.21
1844-12-31,-9.81


In [9]:
#merge the TMAX and TMIN datasets by YEAR
df = pd.merge(tmax_df, tmin_df, on="date_time")

In [10]:
df.head()

,TMAX,TMIN
date_time,,
1840-12-31,23.74,-8.31
1841-12-31,25.14,-9.21
1842-12-31,24.74,-7.11
1843-12-31,24.84,-12.21
1844-12-31,25.14,-9.81


### Push results to MySQL

In [12]:
# Import SQL Alchemy
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [13]:
rds_connection_string = "pythonuser:password@127.0.0.1/global_temp_sea_ice_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [14]:
# Confirm tables
engine.table_names()

['global_temp', 'sea_ice']

In [15]:
#push df to sql
df.to_sql(name='global_temp', con=engine, if_exists='append', index=True)

In [16]:
#confirm data has been successfully pushed to mySQL
pd.read_sql_query('select * from global_temp', con=engine).head()

,date_time,tmax,tmin
0,1840-12-31,23.74,-8.31
1,1841-12-31,25.14,-9.21
2,1842-12-31,24.74,-7.11
3,1843-12-31,24.84,-12.21
4,1844-12-31,25.14,-9.81
